In [1]:
! pip install numpy
! pip install networkx
! pip install pgmpy
! pip install ipdb
! pip install yfinance
! pip install matplotlib
! pip install arviz
! pip install pymc
! pip install bambi
! pip install pyreadline



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip

[notice] 

In [117]:
import networkx
import pickle
import numpy as np
from stocks import Stocks, Ticker
import arviz as az
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pymc as pm
import pickle
import matplotlib.pyplot as plt
from pprint import pprint
from stocks import Stocks, Ticker
import bambi as bmb
import xarray as xr
from StockEngine import StockEngine
from tqdm import tqdm
import ipdb

%config IPCompleter.greedy=True


In [118]:
def predictions_vol_file(tday):
    return f"predictions/{tday}_vol.pkl"

def predictions_corr_file(tday):
    return f"predictions/{tday}_corr.pkl"

with open("adj_mat.pkl", "rb") as f:
    G = pickle.load(f)
    
G.remove_node('DVA')
G.remove_node('ULTA')

tickers = list(G.nodes())

# UNCOMMENT IN FUTURE
# tickers = tickers[:5]

ticker_to_index = {}
for i, ticker in enumerate(tickers):
    ticker_to_index[ticker] = i
print(tickers)
print(ticker_to_index)
num_tickers = len(tickers)
corr_len = num_tickers * (num_tickers - 1) // 2

start_date = "2010-01-01"
end_date = "2021-01-01"

prediction_folder="predictions"

engine = StockEngine(tickers, start_date=start_date, end_date=end_date)

['CTSH', 'CI', 'CMS', 'ROST', 'JNJ', 'PFG', 'PAYX', 'AON', 'FIS', 'DXC', 'ROK', 'AMZN', 'ISRG', 'PNR', 'GWW', 'LNT', 'CE', 'NKE', 'MCO', 'AEP', 'TPR', 'UNH', 'NVR', 'XEL', 'TTWO', 'CMG', 'GIS', 'WST', 'CAT', 'XRAY', 'ANSS', 'QCOM', 'INTC', 'LIN', 'MU', 'GPC', 'ITW', 'EL', 'DXCM', 'COO', 'PKG', 'AMD', 'ETN', 'MHK', 'IEX', 'HON', 'MKC', 'MO', 'BIO', 'DIS', 'NFLX', 'RL', 'CSX', 'T', 'WM', 'TER', 'CRL', 'CL', 'JKHY', 'USB', 'COF', 'TFC', 'GS', 'AVB', 'UDR', 'ESS', 'HIG', 'PEAK', 'SYY', 'WAT', 'BWA', 'NEE', 'PPL', 'LLY', 'INCY', 'LVS', 'COST', 'XOM', 'SLB', 'DVN', 'WMB', 'PXD', 'EOG', 'PSA', 'TSCO', 'TAP']
{'CTSH': 0, 'CI': 1, 'CMS': 2, 'ROST': 3, 'JNJ': 4, 'PFG': 5, 'PAYX': 6, 'AON': 7, 'FIS': 8, 'DXC': 9, 'ROK': 10, 'AMZN': 11, 'ISRG': 12, 'PNR': 13, 'GWW': 14, 'LNT': 15, 'CE': 16, 'NKE': 17, 'MCO': 18, 'AEP': 19, 'TPR': 20, 'UNH': 21, 'NVR': 22, 'XEL': 23, 'TTWO': 24, 'CMG': 25, 'GIS': 26, 'WST': 27, 'CAT': 28, 'XRAY': 29, 'ANSS': 30, 'QCOM': 31, 'INTC': 32, 'LIN': 33, 'MU': 34, 'GPC': 3

100%|███████████████████████████████████████████████████████████████| 86/86 [00:00<00:00, 499.21it/s]


In [119]:
import datetime

num_prev_trading_days = 126
months_to_evaluate = 12 # * 10
evaluation_year_month_start = "2012-01"

In [120]:
def convert_to_lower_triangular(A, dimension):
    C = np.zeros((A.shape[0], dimension, dimension))
    B = np.zeros((dimension, dimension))
    idx = np.tril_indices_from(B, -1)
    
    for i in range(len(A)):
        C[i][idx] = A[i]
    return C

In [121]:
def reconstruct_coefficients_vol(vol, ticker):
    return (
        vol[ticker].posterior['alpha'][0][-1].to_numpy(),
        vol[ticker].posterior['betas_day'][0][-1].to_numpy(),
        vol[ticker].posterior['betas_week'][0][-1].to_numpy(),
        vol[ticker].posterior['betas_month'][0][-1].to_numpy(),
    )

def reconstruct_coefficients_corr(corr):
    return (
        corr.posterior['alpha'][0][-1].to_numpy(),
        corr.posterior['betas_day'][0][-1].to_numpy(),
        corr.posterior['betas_week'][0][-1].to_numpy(),
        corr.posterior['betas_month'][0][-1].to_numpy(),
    )

In [122]:
def make_prediction(tday, vol, corr):
    _, RVt1, RVt25, RVt626, _, xt1, xt25, xt626 = engine.get_HAR_DRD_data(tday)
    
    # Reconstruct RV_t
    predicted_RV_t = np.zeros(num_tickers)
    for ticker_index in range(num_tickers):
        ticker = tickers[ticker_index]
        
        neighbors = list(G.adj[ticker].keys())
        alpha, betas_day, betas_week, betas_month = vol[ticker]
        
        predicted_RV_t[ticker_index] += alpha
        predicted_RV_t[ticker_index] += betas_day[0] * RVt1[ticker_index]
        predicted_RV_t[ticker_index] += betas_week[0] * RVt25[ticker_index]
        predicted_RV_t[ticker_index] += betas_month[0] * RVt626[ticker_index]
        
        for i, neighbor_ticker in enumerate(neighbors):
            neighbor_ticker_index = ticker_to_index[neighbor_ticker]
            predicted_RV_t[ticker_index] += betas_day[i+1] * RVt1[neighbor_ticker_index] 
            predicted_RV_t[ticker_index] += betas_week[i+1] * RVt25[neighbor_ticker_index] 
            predicted_RV_t[ticker_index] += betas_month[i+1] * RVt626[neighbor_ticker_index] 
    
    
    # Reconstruct R_t
    alpha, betas_day, betas_week, betas_month = corr[ticker]
    predicted_R_t = alpha + betas_day * xt1 + betas_week * xt25 + betas_month * xt626
    
#     print("predicted RV_t", predicted_RV_t)
#     print("predicted R_t", predicted_R_t)
    
    D_t = np.diag(np.sqrt(predicted_RV_t))
#    D_t = np.diag(np.sqrt(np.maximum(predicted_RV_t, 0)))
    R_t = np.zeros((num_tickers, num_tickers))
    R_t[np.tril_indices_from(R_t, -1)] = predicted_R_t
    
    H_t = D_t @ R_t @ D_t
    
    return H_t
    
# Old code under the assumption we had unique coeff for each :3
#     predicted_R_t = np.zeros(corr_len)
#     indices = np.tril_indices_from(np.zeros((num_tickers, num_tickers)))
#     indices_list = list(zip(new_idx[0], new_idx[1]))
#     for idx, (ticker_1_index, ticker_2_index) in enumerate(indices_list):
#         alpha, betas_day, betas_week, betas_month = reconstruct_coefficients(vol, ticker)
        
#         predicted_R_t[idx] += 
        
    

In [123]:
def loss_frobenius(cov1, cov2):
    return np.linalg.norm(cov1 - cov2, ord='fro')

In [127]:
losses = []
USE_YEARLY = True
for month_diff in range(months_to_evaluate):
    if USE_YEARLY:
        year_month = engine.get_shift_month(evaluation_year_month_start, month_diff * 12)
    else:
        year_month = engine.get_shift_month(evaluation_year_month_start, month_diff) 
        
    start_day = 1
    tday = year_month + "-{:02d}".format(start_day)
    while tday not in engine.trading_day_list:
        start_day += 1
        tday = year_month + "-{:02d}".format(start_day) 

    with open(predictions_vol_file(tday), "rb") as f:
        vol = pickle.load(f)
    with open(predictions_corr_file(tday), "rb") as f:
        corr = pickle.load(f)
        
    predicted_H_t = make_prediction(tday, vol, corr)
    H_t, _, _, _ = engine.get_HAR_DRD(tday)
    loss = loss_frobenius(predicted_H_t, H_t)
    print(f"{tday} Loss", loss)
#     print("Predicted H_t", predicted_H_t)
#     print("H_t", H_t)
    with open("evaluation/{tday}_evaluation.pkl", "wb") as f:
        pickle.dump({"predicted_H_t" : predicted_H_t,
                "H_t" : H_t,
                "loss": loss
               }, f)
    losses.append(loss)
    
print(losses)

2012-01-03 Loss 0.4560691584233638
2013-01-02 Loss 0.2022804680405461
2014-01-02 Loss 0.1453416252852822
2015-01-02 Loss 0.18112197536946795


FileNotFoundError: [Errno 2] No such file or directory: 'predictions/2016-01-04_vol.pkl'

In [ ]:
make_prediction("2012-01-03", vol, corr)